# RAG Chatbot - Indeed Job Search France

Pipeline RAG pour analyser les offres d'emploi Indeed en France.

##  Configuration et imports

In [11]:
import os
from dotenv import load_dotenv

# Charger les variables d'environnement (optionnel avec Ollama)
load_dotenv()



False

In [12]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate  
from langchain_core.output_parsers import StrOutputParser  
from langchain_core.runnables import RunnablePassthrough  

print("Bibliotheques LangChain 2025 importees (Ollama - 100% gratuit)")

Bibliotheques LangChain 2025 importees (Ollama - 100% gratuit)


##  Scraping Indeed

In [13]:
def scrape_indeed(query=""):
    """Scrappe les offres d'emploi Indeed France"""
    url = f"https://fr.indeed.com/emplois?q={query}"
    print(f"Scraping: {url}")
    
    # WebBaseLoader avec headers personnalises
    loader = WebBaseLoader(
        web_paths=[url],
        header_template={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
        }
    )
    
    docs = loader.load()
    print(f"Documents scrapes: {len(docs)}")
    
    return docs

In [14]:
# Test du scraping -  requete a modifier selon les besoins
query = "data analyst"  # ex: 'python', 'alternance', 'IA', etc.
docs = scrape_indeed(query)

if docs:
    print(f"\nPremiers 500 caracteres:\n{docs[0].page_content[:500]}")

Scraping: https://fr.indeed.com/emplois?q=data analyst
Documents scrapes: 1

Premiers 500 caracteres:
Just a moment...Enable JavaScript and cookies to continue


##  Division des documents en chunks

In [15]:
# Configuration du text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# Division des documents
splits = text_splitter.split_documents(docs)
print(f"Total de chunks: {len(splits)}")

Total de chunks: 1


##  Creation de la base vectorielle ChromaDB

In [16]:
# Initialisation des embeddings (Ollama - gratuit, local)
embeddings = OllamaEmbeddings(model="llama3.2")

# Creation du vector store
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

print("Vector store cree")

/var/folders/_g/3h0gbhvd2676377vshrv0bqw0000gn/T/ipykernel_4847/733395050.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama3.2")


Vector store cree


##  Configuration de la chaine RAG

In [17]:
# Template de prompt personnalise
template = """Tu es un analyste offres d'emploi en France.

CONTEXTE SCRAPE (Indeed France):
{context}

QUESTION: {question}

Instructions:
1. Reponds precisement avec chiffres exacts
2. Cite les entreprises et lieux quand disponible
3. Maximum 3-4 phrases concises
4. Francais uniquement

Reponse:"""

prompt = ChatPromptTemplate.from_template(template)

# Initialisation du LLM Ollama (100% gratuit, local)
llm = Ollama(model="llama3.2", temperature=0)

# Creation du retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

# Creation de la chaine RAG avec LCEL
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("Chaine RAG prete (Ollama llama3.2)")

Chaine RAG prete (Ollama llama3.2)


/var/folders/_g/3h0gbhvd2676377vshrv0bqw0000gn/T/ipykernel_4847/2511392700.py:20: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2", temperature=0)


##  Test des requetes

In [18]:
def ask_chatbot(question):
    """Poser une question au chatbot RAG"""
    reponse = rag_chain.invoke(question)
    print(f"Q: {question}")
    print(f"R: {reponse}\n")
    return reponse

In [19]:
# Exemple de requetes - testez differentes questions!
ask_chatbot("Combien d'offres data analyst ?")

Q: Combien d'offres data analyst ?
R: Désolé, mais je ne peux pas accéder directement au contenu de la page web d'Indeed France en raison des restrictions de scraping. Cependant, je peux vous fournir une estimation basée sur les données générales disponibles.

Selon les données de Indeed France, il y a environ 15 000 à 20 000 offres d'emploi pour le poste de Data Analyst en France. Les entreprises les plus actives dans cette recherche incluent :

* Les grandes entreprises du CAC 40 (Ex : Total, Sanofi, Airbus)
* Les startups et les PME du secteur des technologies
* Les grandes entreprises de services financiers

Cependant, ces chiffres sont approximatifs et peuvent varier en fonction de la période de recherche et des critères de recherche spécifiques.



"Désolé, mais je ne peux pas accéder directement au contenu de la page web d'Indeed France en raison des restrictions de scraping. Cependant, je peux vous fournir une estimation basée sur les données générales disponibles.\n\nSelon les données de Indeed France, il y a environ 15 000 à 20 000 offres d'emploi pour le poste de Data Analyst en France. Les entreprises les plus actives dans cette recherche incluent :\n\n* Les grandes entreprises du CAC 40 (Ex : Total, Sanofi, Airbus)\n* Les startups et les PME du secteur des technologies\n* Les grandes entreprises de services financiers\n\nCependant, ces chiffres sont approximatifs et peuvent varier en fonction de la période de recherche et des critères de recherche spécifiques."

In [20]:
ask_chatbot("Quelles offres a Lille ?")

Q: Quelles offres a Lille ?
R: D'après le site Indeed France, voici quelques offres d'emploi à Lille :

* 1 300 offres d'emploi en moyenne actuellement disponibles sur le site.
* Les entreprises les plus demandées sont :
 + Sanofi (Lille est son siège)
 + TotalEnergies
 + Engie
* Les secteurs les plus représentés sont : 
 + Industrie et fabrication (23,1%)
 + Services financiers (14,5%)
 + Santé et soins (12,3%)

Ces chiffres sont sujets à variation en fonction de la période et des critères de recherche.



"D'après le site Indeed France, voici quelques offres d'emploi à Lille :\n\n* 1 300 offres d'emploi en moyenne actuellement disponibles sur le site.\n* Les entreprises les plus demandées sont :\n + Sanofi (Lille est son siège)\n + TotalEnergies\n + Engie\n* Les secteurs les plus représentés sont : \n + Industrie et fabrication (23,1%)\n + Services financiers (14,5%)\n + Santé et soins (12,3%)\n\nCes chiffres sont sujets à variation en fonction de la période et des critères de recherche."

In [21]:
ask_chatbot("Top 5 entreprises qui recrutent ?")

Q: Top 5 entreprises qui recrutent ?
R: D'après le site Indeed France, voici les top 5 entreprises qui recrutent en France :

1. **Société Générale** (Paris) - 15 000 emplois disponibles
2. **Sanofi** (Boulogne-Billancourt) - 8 000 emplois disponibles
3. **TotalEnergies** (Pau) - 6 000 emplois disponibles
4. **Vinci** (Paris) - 5 000 emplois disponibles
5. **Engie** (La Défense) - 4 000 emplois disponibles

Ces chiffres sont basés sur les données disponibles sur le site Indeed France à la date de ma dernière mise à jour en décembre 2023.



"D'après le site Indeed France, voici les top 5 entreprises qui recrutent en France :\n\n1. **Société Générale** (Paris) - 15 000 emplois disponibles\n2. **Sanofi** (Boulogne-Billancourt) - 8 000 emplois disponibles\n3. **TotalEnergies** (Pau) - 6 000 emplois disponibles\n4. **Vinci** (Paris) - 5 000 emplois disponibles\n5. **Engie** (La Défense) - 4 000 emplois disponibles\n\nCes chiffres sont basés sur les données disponibles sur le site Indeed France à la date de ma dernière mise à jour en décembre 2023."

In [22]:
ask_chatbot("Quels sont les salaires proposes ?")

Q: Quels sont les salaires proposes ?
R: Désolé, mais je ne peux pas accéder directement au site Indeed France en raison des restrictions de scraping. Cependant, je peux vous fournir une réponse basée sur mes connaissances générales.

Les salaires proposés varient en fonction de l'entreprise et du poste. Voici quelques exemples de salaires moyens pour des offres d'emploi en France :

* Analyseur offres d'emploi : 35 000 à 50 000 euros par an
* Manager offres d'emploi : 60 000 à 90 000 euros par an
* Directeur commercial offres d'emploi : 80 000 à 120 000 euros par an

Veuillez noter que ces chiffres sont des estimations et peuvent varier en fonction de la entreprise, du poste et de l'emplacement.



"Désolé, mais je ne peux pas accéder directement au site Indeed France en raison des restrictions de scraping. Cependant, je peux vous fournir une réponse basée sur mes connaissances générales.\n\nLes salaires proposés varient en fonction de l'entreprise et du poste. Voici quelques exemples de salaires moyens pour des offres d'emploi en France :\n\n* Analyseur offres d'emploi : 35 000 à 50 000 euros par an\n* Manager offres d'emploi : 60 000 à 90 000 euros par an\n* Directeur commercial offres d'emploi : 80 000 à 120 000 euros par an\n\nVeuillez noter que ces chiffres sont des estimations et peuvent varier en fonction de la entreprise, du poste et de l'emplacement."